In [6]:
import os
import joblib # ou pickle

# --- BLOC DE CONFIGURATION DES CHEMINS (À METTRE AU DÉBUT) ---

# Le chemin vers le dossier 'models' est UN NIVEAU AU-DESSUS (../) du dossier 'notebooks'
MODELS_DIR = '../models'

# On s'assure que ce dossier existe. S'il n'existe pas, on le crée.
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

# -----------------------------------------------------------------

In [7]:
# ==============================================================================
# NOTEBOOK FINAL ET UNIQUE (Version 4 - Anti-Crash)
# ==============================================================================
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib
import os

# --- ÉTAPE 1: CONNEXION BDD ---
print("--- ÉTAPE 1: Connexion BDD ---")
db_user, db_password, db_host, db_port, db_name = 'root', 'root', 'localhost', '3306', 'sicda_easytime'
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
print("Connexion réussie.")

# --- ÉTAPE 2: CHARGEMENT ET NETTOYAGE ---
print("\n--- ÉTAPE 2: Chargement et nettoyage ---")
df_absences = pd.read_sql("""
    SELECT USER_FK as employe_id, DATE_DEBUT as date_debut, DATE_REPRISE as date_fin 
    FROM absence WHERE STATUT = 'workflow_status_validated'
    UNION ALL
    SELECT USER_FK as employe_id, DATE_DEBUT as date_debut, DATE_REPRISE as date_fin 
    FROM conge WHERE STATUT = 'workflow_status_validated'
""", engine)
df_employes = pd.read_sql("SELECT ID as employe_id, SOLDE_ANNUEL as solde_conges_jours FROM utilisateur", engine)

df_absences['date_debut'] = pd.to_datetime(df_absences['date_debut'], errors='coerce')
df_absences['date_fin'] = pd.to_datetime(df_absences['date_fin'], errors='coerce')
df_absences.dropna(subset=['date_debut', 'date_fin'], inplace=True)
df_absences['duree_absence_jours'] = (df_absences['date_fin'] - df_absences['date_debut']).dt.days + 1
print(f"{len(df_absences)} absences/congés valides chargés.")

# --- ÉTAPE 3: GÉNÉRATION DU DATASET ---
print("\n--- ÉTAPE 3: Génération du dataset d'entraînement ---")
dataset_rows = []
decisions = ['DECOMPTE_SOLDE', 'JUSTIFICATIF_REQUIS', 'AVERTISSEMENT']

for _, absence in df_absences.iterrows():
    employe_id = absence['employe_id']
    solde_info = df_employes[df_employes['employe_id'] == employe_id]
    solde_conges = solde_info['solde_conges_jours'].iloc[0] if not solde_info.empty else 0
    
    decision = np.random.choice(decisions, p=[0.6, 0.3, 0.1])
    if solde_conges is not None and solde_conges <= 1:
        decision = np.random.choice(['JUSTIFICATIF_REQUIS', 'AVERTISSEMENT'], p=[0.5, 0.5])
    
    dataset_rows.append({
        'duree_absence_jours': absence['duree_absence_jours'],
        'solde_conges_jours': solde_conges if solde_conges is not None else 0,
        'nb_absences_injustifiees_annee': np.random.randint(0, 5),
        'est_adjacent_weekend_ferie': 1 if absence['date_debut'].weekday() in [0, 4] else 0,
        'charge_equipe': round(np.random.uniform(0.4, 1.0), 2),
        'decision_absence': decision
    })

df_final = pd.DataFrame(dataset_rows)
print(f"Dataset généré avec {len(df_final)} lignes.")

# ==============================================================================
# NOUVELLE ÉTAPE DE SÉCURITÉ : GARANTIR AU MOINS 2 EXEMPLES PAR CLASSE
# ==============================================================================
print("\n--- ÉTAPE 3.5: Vérification et duplication des classes rares ---")
class_counts = df_final['decision_absence'].value_counts()
print("Distribution des décisions AVANT duplication:\n", class_counts)

# On identifie les classes avec un seul exemple
classes_rares = class_counts[class_counts < 2].index

for classe in classes_rares:
    # On trouve la ligne de cette classe rare
    ligne_a_dupliquer = df_final[df_final['decision_absence'] == classe]
    # On l'ajoute à nouveau au DataFrame
    df_final = pd.concat([df_final, ligne_a_dupliquer], ignore_index=True)
    print(f"La classe '{classe}' a été dupliquée pour éviter une erreur.")

print("\nDistribution des décisions APRÈS duplication:\n", df_final['decision_absence'].value_counts())
# ==============================================================================

# ==============================================================================
# --- ÉTAPE 4: ENTRAÎNEMENT DU MODÈLE (Version Robuste) ---
# ==============================================================================
print("\n--- ÉTAPE 4: Entraînement du modèle ---")

# --- 4.1 Préparation des données pour le modèle ---
encoder_decision_absence = LabelEncoder()
df_final['decision_absence_encoded'] = encoder_decision_absence.fit_transform(df_final['decision_absence'])

features = ['duree_absence_jours', 'solde_conges_jours', 'nb_absences_injustifiees_annee', 'est_adjacent_weekend_ferie', 'charge_equipe']
target = 'decision_absence_encoded'
X = df_final[features]
y = df_final[target]

# --- 4.2 Séparation dynamique et sécurisée des données ---
num_classes = len(y.unique())
n_samples = len(X)

# On s'assure que la taille du test est au minimum le nombre de classes pour permettre la stratification
final_test_size = max(int(n_samples * 0.2), num_classes)

if n_samples > final_test_size:
    print(f"Dataset de {n_samples} échantillons. Séparation en Test ({final_test_size}) et Entraînement.")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=final_test_size, random_state=42, stratify=y
    )
else:
    print(f"ATTENTION: Pas assez de données ({n_samples}) pour créer un ensemble de test valide. Tout sera utilisé pour l'entraînement.")
    X_train, y_train = X, y
    X_test, y_test = pd.DataFrame(columns=X.columns), pd.Series(dtype=y.dtype) # Créer des dataframes vides pour la suite

# --- 4.3 Entraînement du modèle ---
print(f"Nombre de classes détectées : {num_classes}")
lgbm_classifier = lgb.LGBMClassifier(objective='multiclass', num_class=num_classes, random_state=42)
lgbm_classifier.fit(X_train, y_train)
print("Entraînement du modèle terminé.")

# --- 4.4 Évaluation du modèle (seulement si on a un ensemble de test) ---
if not X_test.empty:
    print("\nÉvaluation du modèle...")
    y_pred = lgbm_classifier.predict(X_test)
    print(classification_report(y_test, y_pred, target_names=encoder_decision_absence.classes_, zero_division=0))
else:
    print("\nPas d'évaluation possible car il n'y a pas d'ensemble de test.")

# --- ÉTAPE 5: SAUVEGARDE ---
print("\n--- ÉTAPE 5: Sauvegarde ---")

model_path = os.path.join(MODELS_DIR, 'model_absence_injustifiee.pkl')
joblib.dump(lgbm_classifier, model_path)
print(f"Modèle sauvegardé dans : {model_path}")
encoder_path = os.path.join(MODELS_DIR, 'encoder_decision_absence.pkl')
joblib.dump(encoder_decision_absence, encoder_path)
print(f"Encodeur de décision sauvegardé dans : {encoder_path}")
    
print("\nProcessus terminé.")

--- ÉTAPE 1: Connexion BDD ---
Connexion réussie.

--- ÉTAPE 2: Chargement et nettoyage ---
6 absences/congés valides chargés.

--- ÉTAPE 3: Génération du dataset d'entraînement ---
Dataset généré avec 6 lignes.

--- ÉTAPE 3.5: Vérification et duplication des classes rares ---
Distribution des décisions AVANT duplication:
 decision_absence
DECOMPTE_SOLDE         4
JUSTIFICATIF_REQUIS    2
Name: count, dtype: int64

Distribution des décisions APRÈS duplication:
 decision_absence
DECOMPTE_SOLDE         4
JUSTIFICATIF_REQUIS    2
Name: count, dtype: int64

--- ÉTAPE 4: Entraînement du modèle ---
Dataset de 6 échantillons. Séparation en Test (2) et Entraînement.
Nombre de classes détectées : 2
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train